In [2]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import json
import os
from detectron2.engine import DefaultPredictor
from detectron2.data import build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2 import model_zoo

# Cargar el archivo original en formato lista
with open("/home/toukapy/Documentos/Master CV/C5/mcv-c5-team2/week1/kitti_mots_coco.json", "r") as f:
    original_data = json.load(f)

# Convertir al formato COCO correcto
coco_format = {"images": [], "annotations": [], "categories": []}
ann_id = 1
for idx, item in enumerate(original_data):
    coco_format["images"].append({
        "id": idx,
        "file_name": item["image"]["file_name"],
        "width": item["image"]["width"],
        "height": item["image"]["height"]
    })

    for bbox, category_id in zip(item["objects"]["bbox"], item["objects"]["category"]):
        coco_format["annotations"].append({
            "id": ann_id,
            "image_id": idx,
            "category_id": category_id,
            "bbox": bbox,
            "area": bbox[2] * bbox[3],
            "iscrowd": 0
        })
        ann_id += 1

# Añadir categorías manualmente
coco_format["categories"] = [
    {"id": 1, "name": "car"},
    {"id": 2, "name": "person"}
]

# Guardar en formato válido
output_json = "/home/toukapy/Documentos/Master CV/C5/mcv-c5-team2/week1/kitti_mots_coco_valid.json"
with open(output_json, "w") as f:
    json.dump(coco_format, f)

# Registrar KITTI-MOTS en formato COCO
image_root = "/home/toukapy/Documentos/Master CV/C5/mcv-c5-team2/week1/data/KITTI-MOTS/training/images"
register_coco_instances("kitti_mots_val", {}, output_json, image_root)

# Cargar configuración desde detectron2 model zoo
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # clases KITTI-MOTS: car y person
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.DATASETS.TEST = ("kitti_mots_val",)

# Predictor
predictor = DefaultPredictor(cfg)

# Evaluación usando COCOEvaluator
evaluator = COCOEvaluator("kitti_mots_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "kitti_mots_val")

# Evaluación
print("Evaluando Faster R-CNN en KITTI-MOTS...")
metrics = inference_on_dataset(predictor.model, val_loader, evaluator)

# Mostrar resultados
print(json.dumps(metrics, indent=2))




TypeError: string indices must be integers, not 'str'